In [241]:
lines="""*FRS2881.1.2* Configure Users tab Functionality
# Item 1
## This tab will be selected when the user first navigates to this UI
## When selected, the tab will be larger in size than the other tab
## When not selected, the tab will be smaller in size than the other tab
## When clicked while not selected, it will show the Configure Users tab
# Item 2 See its FRS section for its full spec.
# Item 3
## This will be populated with the user accounts in the PBSUsers.conf file
## Selecting items
### When an item is selected, it will be highlighted blue and its text will be white
### 1 item is to be selected at all times
### No more than 1 item is to be selected at any time
### When the UI is navigated to, the first item will be selected
# Item 4 - Edit User button
## When clicked, this will open the Edit User dialog, and pass it the selected User Name
# Item 5 - Add User button
## When clicked, this will open the Edit User dialog, and pass it a "New User".  See below for further specifications for the Edit User dialog for this condition.
# Item 6 - Delete User button
## When clicked, the selected user will be deleted, and the first user in the list will be selected.  A User Event will be recorded with the message "Deleted User X" with X being the account name of the deleted user and the user who initiated this event.
## This button will be grayed out under the following conditions:
### If there is only 1 user in the User List
### If the selected user in the User List is the current user (i.e. a user won't be able to delete their own account)
# When a user is added, edited or deleted, or a group name is edited, refresh the user list to reflect the changes
"""

In [385]:
import re
start_re = re.compile(r"^([\+\*]* )?([\+\* ]*)(FRS\d+)\.?([\d\.]*)([\+\*\:])")

class BadFormat(ValueError):
    pass

def new_start(m):
    indent, fmt, frs, nums, _ = m.groups()
    lvl = len(fmt.replace(" ", ""))
    start = "".join((fmt, frs, ".%s", fmt[::-1], ":"))
    return len(indent or ""), start, nums

def parse_indent(l):
    initial, frag = l.split(" ", 1)
    c = initial[0]
    if c not in "*#":
        lvl = 0
    else:
        lvl = len(initial)
        
        # sanity check
        if lvl != initial.count(c):
            raise BadFormat(l)
    return lvl, frag


def istate_maybe_extend(istate, new, initial=1):
    extend = new - len(istate) + 1
    if extend > 0:
        istate.extend(initial for _ in range(extend))
        

def istate_update_level(istate, old, new):
    istate_maybe_extend(istate, new, 1)
    if new > old:
        for s in range(old+1, len(istate)):
            istate[s] = 1
    elif new < old:
        istate[new] += 1
    else:
        istate [new] += 1
    #print(istate[:new+1])
        
def istate_new_start(istate, nums):
    if nums:
        lnums = nums.split(".")
    else:
        lnums = [1]
    istate_maybe_extend(istate, len(lnums), 0)
    for i, c in enumerate(lnums):
        n = int(c)
        istate[i] = n
    return i
    
        
def ignore_match(ignore, istate):
    if ignore is None:
        return False
    for ig in ignore:
        for a, b in zip(ig, istate):
            if a != b:
                return False
    return True

def parse_text(t, depth=1, ignore=None):
    lines = t.splitlines()
    start = None
    istate = [1]
    out = []
    clvl = lvl = offset = 0
    for l in lines:
        m = start_re.match(l)
        if m:
            ilvl, start, nums = new_start(m)
            clvl = istate_new_start(istate, nums)
            offset = clvl - ilvl
            out.append(l)
            continue
        
        if not (l.startswith("#") or l.startswith("*")) or ignore_match(ignore, istate):
            out.append(l)
            continue

        lvl, frag = parse_indent(l)
        #print(istate)#[:lvl + offset+1])
        if lvl <= depth:
            istate_update_level(istate, clvl+offset, lvl+offset)
            num = ".".join(str(i) for i in istate[:lvl+offset+1])
            
            pre = "*" * lvl
            if pre:
                pre += " "
            op = "".join((pre, start % num, " ", frag))
            out.append(op)
        else:
            out.append(("*" * lvl + " " + frag))
        clvl = lvl
    return out

In [386]:
print("\n".join(parse_text(lines, 3)))

*FRS2881.1.2* Configure Users tab Functionality
* *FRS2881.1.2.1*: Item 1
** *FRS2881.1.2.1.1*: This tab will be selected when the user first navigates to this UI
** *FRS2881.1.2.1.2*: When selected, the tab will be larger in size than the other tab
** *FRS2881.1.2.1.3*: When not selected, the tab will be smaller in size than the other tab
** *FRS2881.1.2.1.4*: When clicked while not selected, it will show the Configure Users tab
* *FRS2881.1.2.2*: Item 2 See its FRS section for its full spec.
* *FRS2881.1.2.3*: Item 3
** *FRS2881.1.2.3.1*: This will be populated with the user accounts in the PBSUsers.conf file
** *FRS2881.1.2.3.2*: Selecting items
*** *FRS2881.1.2.3.2.1*: When an item is selected, it will be highlighted blue and its text will be white
*** *FRS2881.1.2.3.2.2*: 1 item is to be selected at all times
*** *FRS2881.1.2.3.2.3*: No more than 1 item is to be selected at any time
*** *FRS2881.1.2.3.2.4*: When the UI is navigated to, the first item will be selected
* *FRS2881.1.

In [246]:
import re
frs_match = re.compile(r"^(?:([\+\*]*) )?([\+\* ]*)(FRS\d+)\.?([\d\.]+)?([\+\*\:]*)?\s*(.+)?$").match
subitem_match = re.compile(r"^([#*] )")

class NoMatch(Exception):
    pass

class NumState():
    VALID_INDENTS = "*+"
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    def __init__(self, startval=1, depth=2, indent_char="*"):
        self._startval = startval
        self.state = self._newstate()
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
    def _newstate(self):
        return [self._startval]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
    
    def set_state(self, lvl, val):
        self._grow(lvl)
        self.state[lvl] = val
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def set_str(self, s):
        m = frs_match(s)
        if not m:
            raise NoMatch(s)
        return self.set_match(m)

    def set_match(self, m):
        indent, pre_fmt, frs, nums, post_fmt, bob = m.groups()
        self.ilvl = len(indent or "")
        self.fmt = pre_fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        
    def frs_string(self):
        return self.format_lvl(self.clvl)
        
    def format_lvl(self, lvl):
        if lvl < 0:
            lvl = 0
        ind = self.indent_char * (lvl + self.ilvl)
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.state[:self.nlvl + lvl])
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.state[self.clvl] += 1
        
    def next(self):
        self.state[self.clvl+self.nlvl - 1] += 1
        
    def feed(self, line):
        self.line = line
        
        m = frs_match(line)
        if m:
            self.set_match(m)
            return self.LINE_FRS
        m = subitem_match(line)
        if m:
            return self.LINE_SUBITEM
        return self.LINE_OTHER
            

In [247]:
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")

In [248]:
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")

In [249]:
s = "* *FRS1234*"; state = NumState(); state.set_str(s); state.indent()
def pcall(f):
    try:
        return f()
    except Exception as e:
        return "Error: %s" % e.args
print(pcall(lambda: assert_fmt(s, 1, "** *FRS1234.1:*")))
print(pcall(lambda: assert_next("*FRS1234.1*", "*FRS1234.2:*")))

None
None


In [250]:
def test_match(s):
    return frs_match(s).groups()

def assert_match(s, exp):
    m = test_match(s)
    assert m == exp, (m, exp)
    
assert_match("*FRS1234*", (None, "*", "FRS1234", None, "*", None))
assert_match("* *FRS1234*", ("*", "*", "FRS1234", None, "*", None))
assert_match("*FRS1234.5.6*", (None, "*", "FRS1234", "5.6", "*", None))
assert_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234", "5.6", "+*", None))
assert_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234", "5.6.7.8", "+*", None))
assert_match("* *FRS1234:*", ("*", "*", "FRS1234", None, ":*", None))
assert_match("* *FRS1234*:", ("*", "*", "FRS1234", None, "*:", None))
assert_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", None, "*:", "Bob"))

def assert_ab(a, b):
    try:
        assert a == b, "%s != %s" % (a, b)
    except AssertionError as e:
        print(e.args)
        raise

def test_state(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    st.set_str(s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    st.set_str(s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

In [251]:
def parse_text(t):
    lines = t.splitlines()
    state = NumState()
    for line in lines:
        typ = state.feed(line)
        if typ == state.LINE_FRS:
            v = line
        elif typ == state.LINE_SUBITEM:
            fmt = state.frs_string()
            v = " ".join((fmt, line.split(" ", 1)[1]))
        elif typ == state.LINE_OTHER:
            v = line
        else:
            raise ValueError(typ)
        yield v

In [252]:
parsed = parse_text(lines)
for line in parsed:
    print(line)

*FRS2881.1.2* Configure Users tab Functionality


TypeError: '_sre.SRE_Pattern' object is not callable